In [23]:
import os
from openai import OpenAI

client = OpenAI()


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
OpenAI.api_key = os.environ['OPENAI_API_KEY']

## we are making a call directly to openai api

In [25]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [26]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(model=model,
    messages=messages,
    temperature=0)
    return response.choices[0].message.content


In [27]:
get_completion("what is the capital of spain?")


'The capital of Spain is Madrid.'

In [28]:
bday_mone = """
feliz cumle mama. te quiero mucho. todo bien por aqui
"""

In [29]:
style = """galician \
emotional and family style
"""

In [30]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{bday_mone}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is galician emotional and family style
.
text: ```
feliz cumle mama. te quiero mucho. todo bien por aqui
```



In [31]:
response = get_completion(prompt)

In [32]:
print(response)

Feliz aniversario, mamá. Querémoste moito. Todo ben por aquí.


## we are making apicalls using langchain

In [30]:
from langchain_openai import ChatOpenAI


In [31]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fb27ad2ac10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fb27ad3d790>, root_client=<openai.OpenAI object at 0x7fb27b0d9cd0>, root_async_client=<openai.AsyncOpenAI object at 0x7fb27ad2ac40>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [32]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [33]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [34]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [35]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [36]:
customer_style = """American English \
in a calm and respectful tone
"""

In [37]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [38]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [39]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [41]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [42]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


In [43]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [44]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [47]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [48]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a tough break, indeed! Fare thee well!
